#Jeyasree Shanmugaveloo (SN01083033)
#NAVENITHA A/P JEGANATH (SW01082804)

In [5]:
# Load dataset 
import pandas as pd 
file_path = "Reviews.csv" 
df = pd.read_csv(file_path) 
# Display column content without truncation 
pd.set_option('display.max_colwidth', None)  # Set to None for unlimited width 
print(df.head())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary  \
0  Good Quality Dog Food   
1      Not as Advertised   
2  "Delight" says it all   
3         Cough Medicine   
4            Great

In [7]:
text_df = pd.DataFrame()
text_df['text'] = df['Text'].copy()
text_df['score'] =df['Score'].copy()
text_df.head()

,text,score
0,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.,5
1,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo"".",1
2,"This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' ""The Lion, The Witch, and The Wardrobe"" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.",4
3,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.,2
4,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal.",5


In [9]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

text_df['text'] = text_df['text'].apply(clean_text)

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def token_lematize(text):
    # Tokenize
    words = word_tokenize(text)

    # Remove stopwords & lemmatize
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]

    return " ".join(words)
text_df['text'] = text_df['text'].apply(token_lematize)

In [11]:
print(text_df.head())

                                                                                                                                                                                                                                                                                     text  \
0                                                                                                                       bought several vitality canned dog food product found good quality product look like stew processed meat smell better labrador finicky appreciates product better   
1                                                                                                                                                 product arrived labeled jumbo salted peanutsthe peanut actually small sized unsalted sure error vendor intended represent product jumbo   
2  confection around century light pillowy citrus gelatin nut case filbert cut tiny square liberally coated powdered sugar tiny mouthful heaven c

In [13]:
def label_sentiment(score):
    if score >= 4:
        return 'positive'
    elif score <= 2:
        return 'negative'
    else:
        return 'neutral'  # score == 3
text_df['label'] = text_df['score'].apply(label_sentiment)
text_df.tail()

,text,score,label
568449,great sesame chickenthis good better resturants eaten atmy husband loved itwill find recipe use,5,positive
568450,im disappointed flavor chocolate note especially weak milk thickens flavor still disappoints worth try ill never buy use whats left gone time thanks small can,2,negative
568451,star small give one training session tried train dog ceaser dog treat made puppy hyper compare ingredient know little star basic food ingredient without preservative food coloring sweet potato flavor also make hand smell like dog food,5,positive
568452,best treat training rewarding dog good grooming lower calorie loved doggy sweet potato seem favorite wet nose treat,5,positive
568453,satisfied product advertised use cereal raw vinegar general sweetner,5,positive


In [17]:
!pip install textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 7.7 MB/s eta 0:00:00


In [21]:
!pip install vaderSentiment

In [23]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER analyzer
vader_analyzer = SentimentIntensityAnalyzer()

# VADER analysis: compound score and sentiment label
text_df['vader_score'] = text_df['text'].apply(lambda x: vader_analyzer.polarity_scores(x)['compound'])

text_df['vader_sentiment'] = text_df['vader_score'].apply(
    lambda score: 'positive' if score > 0.05 else 'negative' if score < -0.05 else 'neutral'
)

# TextBlob analysis: polarity score and sentiment label
text_df['textblob_polarity'] = text_df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

text_df['textblob_sentiment'] = text_df['textblob_polarity'].apply(
    lambda score: 'positive' if score > 0.05 else 'negative' if score < -0.05 else 'neutral'
)
text_df.tail()

,text,score,label,vader_score,vader_sentiment,textblob_polarity,textblob_sentiment
568449,great sesame chickenthis good better resturants eaten atmy husband loved itwill find recipe use,5,positive,0.9300,positive,0.675000,positive
568450,im disappointed flavor chocolate note especially weak milk thickens flavor still disappoints worth try ill never buy use whats left gone time thanks small can,2,negative,-0.7841,negative,-0.250000,negative
568451,star small give one training session tried train dog ceaser dog treat made puppy hyper compare ingredient know little star basic food ingredient without preservative food coloring sweet potato flavor also make hand smell like dog food,5,positive,0.8860,positive,-0.021875,neutral
568452,best treat training rewarding dog good grooming lower calorie loved doggy sweet potato seem favorite wet nose treat,5,positive,0.9738,positive,0.521429,positive
568453,satisfied product advertised use cereal raw vinegar general sweetner,5,positive,0.4215,positive,0.106410,positive


In [25]:
from sklearn.metrics import classification_report
# TextBlob classification report
tb_report = classification_report(
    y_true=text_df['label'],
    y_pred=text_df['textblob_sentiment'],
    target_names=['negative', 'neutral', 'positive']
)

# VADER classification report
vader_report = classification_report(
    y_true=text_df['label'],
    y_pred=text_df['vader_sentiment'],
    target_names=['negative', 'neutral', 'positive']
)

# Print reports
print("\n📊 Classification Report for TextBlob:")
print(tb_report)

print("\n📊 Classification Report for VADER:")
print(vader_report)


📊 Classification Report for TextBlob:
              precision    recall  f1-score   support

    negative       0.56      0.28      0.37     82037
     neutral       0.12      0.15      0.13     42640
    positive       0.85      0.90      0.87    443777

    accuracy                           0.75    568454
   macro avg       0.51      0.44      0.46    568454
weighted avg       0.75      0.75      0.74    568454


📊 Classification Report for VADER:
              precision    recall  f1-score   support

    negative       0.56      0.29      0.38     82037
     neutral       0.12      0.03      0.05     42640
    positive       0.82      0.96      0.89    443777

    accuracy                           0.79    568454
   macro avg       0.50      0.43      0.44    568454
weighted avg       0.73      0.79      0.75    568454



In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(text_df['text'])
y = text_df['label']

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [29]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)

In [31]:
# Linear SVC
svc = LinearSVC()
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)

In [33]:
# Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)

In [35]:
#Classification Reports
print("\nLogistic Regression:\n", classification_report(y_test, logreg_pred,target_names=['negative', 'neutral', 'positive']))
print("\nLinear SVC:\n", classification_report(y_test, svc_pred,target_names=['negative', 'neutral', 'positive']))
print("\nMultinomial Naive Bayes:\n", classification_report(y_test, nb_pred,target_names=['negative', 'neutral', 'positive']))
# Print reports
print("\nClassification Report for TextBlob:")
print(tb_report)
print("\nClassification Report for VADER:")
print(vader_report)


Logistic Regression:
               precision    recall  f1-score   support

    negative       0.73      0.66      0.70     24278
     neutral       0.52      0.18      0.27     12850
    positive       0.90      0.97      0.93    133409

    accuracy                           0.87    170537
   macro avg       0.72      0.60      0.63    170537
weighted avg       0.84      0.87      0.85    170537


Linear SVC:
               precision    recall  f1-score   support

    negative       0.72      0.67      0.70     24278
     neutral       0.57      0.11      0.18     12850
    positive       0.89      0.97      0.93    133409

    accuracy                           0.86    170537
   macro avg       0.73      0.58      0.60    170537
weighted avg       0.84      0.86      0.84    170537


Multinomial Naive Bayes:
               precision    recall  f1-score   support

    negative       0.84      0.26      0.40     24278
     neutral       0.49      0.00      0.00     12850
    positiv

Logistic Regression and Linear SVC deliver the best overall performance, achieving accuracies of 87% and 86% respectively, and showing strong results on the positive sentiment class. However, both models struggle significantly with the neutral class, with recall dropping to just 18% for Logistic Regression and 11% for Linear SVC. Naive Bayes, while simple and efficient, performs poorly on negative and neutral sentiments, tending to overpredict the positive class. Among the lexicon-based approaches, VADER outperforms TextBlob, showing higher accuracy and better recall for positive sentiments, but both fall short in accurately detecting negative and neutral reviews. In general, machine learning models outperform lexicon-based ones, but identifying neutral sentiment remains a consistent challenge across all methods.